In [2]:
# Imports
import json
from collections import OrderedDict

## Definitions

### Transitions Definition

In [6]:
# Json input
input_str= """
{
    "T1":[
        ["P1", "P2"],
        [
            [ ["I1", "I2"], ["I4"] ],
            [ ["I3"], ["I4"] ]
        ],
        ["P3"]
    ],
    "T2":[
        ["P3"],
        [
            [ ["I4"], ["I1", "I2"] ],
            [ ["I4"], ["I3"] ]
        ],
        ["P1", "P2"]
    ]
}
"""

In [8]:
# Convert Json string to python dictionary
input_dict= json.loads(input_str, object_pairs_hook=OrderedDict)

In [9]:
# Expected NuSMV
expected= "T1:= (P1 & P2) & ( (I1 & I2 & !I4) | (I3 & !I4) ) & (!P3);\n" + \
"T2:= (P3) & ( (I4 & !I1 & !I2) | (I4 & !I3) ) & (!P1 & !P2);\n"  

In [35]:
def transition_definition(in_dict):
    """Define transition firing conditions in NuSMV

    Parameters:
    ----------
    in_dict: dict

    Returns:
    -------
    out_str: string
    """
    out_str= ""

    for key in iter(in_dict):

        # Get value of current transition
        value= in_dict[key]

        # Pre-places
        pre= value[0]
        # Pre-places condition
        pre_str= "("
        for idx,item in enumerate(pre):
            # Not last element
            if idx < len(pre)-1:
                pre_str= pre_str + item + " & "
            else:
                pre_str= pre_str + item

        pre_str= pre_str + ")"

        # Inputs
        req_inputs= value[1]
        # Required inputs condition
        req_str= "( "
        for idx_out,item in enumerate(req_inputs):
            high_ins= item[0]
            low_ins= item[1]
            # Temp string opening brace
            temp_str= "("
            # temp inputs
            for idx_high,high_in in enumerate(high_ins):
                # Not last item
                if idx_high < len(high_ins)-1:
                    temp_str= temp_str + high_in + " & "
                # If last item
                else:
                    # If there are no low outputs required
                    if len(low_ins) < 1:
                        temp_str= temp_str + high_in
                    else:
                        temp_str= temp_str + high_in + " & "
            
            # Low inputs
            for idx_low,low_in in enumerate(low_ins):
                # Not last item
                if idx_low < len(low_ins)-1:
                    temp_str= temp_str + "!" + low_in + " & "
                # If last item
                else:
                    temp_str= temp_str + "!" + low_in

            # Temp string closing brace
            temp_str= temp_str + ")"   

            # If not last item, add an OR operator
            if idx_out < (len(req_inputs)-1):
                temp_str= temp_str + " | "

            # Append Temp string to req_str
            req_str= req_str + temp_str

        # req_str closin brace
        req_str= req_str + " )"

        # Post-places
        post= value[2]
        # Post-places condition
        post_str= "("
        for idx_post,item_post in enumerate(post):
            # Not last element
            if idx_post < len(post)-1:
                post_str= post_str + "!" + item_post + " & "
            else:
                post_str= post_str + "!" + item_post

        post_str= post_str + ")"

        # Append transition name to output
        out_str= out_str + key + ":= "
        # Append pre-place condition
        out_str= out_str + pre_str + " & "
        # Append inputs condition
        out_str= out_str + req_str + " & "
        # Append post-place condition
        out_str= out_str + post_str + ";\n"

    return out_str



In [37]:
transition_definition(input_dict)

'T1:= (P1 & P2) & ( (I1 & I2 & !I4) | (I3 & !I4) ) & (!P3);\nT2:= (P3) & ( (I4 & !I1 & !I2) | (I4 & !I3) ) & (!P1 & !P2);\n'

In [38]:
expected

'T1:= (P1 & P2) & ( (I1 & I2 & !I4) | (I3 & !I4) ) & (!P3);\nT2:= (P3) & ( (I4 & !I1 & !I2) | (I4 & !I3) ) & (!P1 & !P2);\n'

### Stability Definition

In [39]:
# Json input
input_str= """
{
    "T1":[
        ["P1", "P2"],
        [
            [ ["I1", "I2"], ["I4"] ],
            [ ["I3"], ["I4"] ]
        ],
        ["P3"]
    ],
    "T2":[
        ["P3"],
        [
            [ ["I4"], ["I1", "I2"] ],
            [ ["I4"], ["I3"] ]
        ],
        ["P1", "P2"]
    ]
}
"""

In [40]:
# Convert Json string to python dictionary
input_dict= json.loads(input_str, object_pairs_hook=OrderedDict)

In [49]:
# Expected NuSMV
expected= "stab:= !(T1 | T2);\n"

In [46]:
def stab_definition(in_dict):
    """Define stability checking condition in NuSMV

    Parameters:
    ----------
    in_dict: dict

    Returns:
    -------
    out_str: string
    """
    out_str="stab:= !("

    keys= list(in_dict)
    for idx,key in enumerate(keys):
        # If not last item
        if idx < len(keys)-1:
            # Append transition name and add OR
            out_str= out_str + key + " | "
        else:
            # Append transition name
            out_str= out_str + key

    # Add closing brace and semicolon
    out_str= out_str + ");\n"

    return out_str


In [47]:
stab_definition(input_dict)

'stab:= !(T1 | T2);\n'

In [50]:
expected

'stab:= !(T1 | T2);\n'

### Output Definition

In [1]:
# Json input
input_str= """
{
    "P1": [
        ["O1", "O2"],
        ["O3"]
    ],
    "P2":[
        ["O2", "O3"],
        ["O1", "O2"]
    ],
    "P3": [
        ["O1"],
        ["O2", "O3"]
    ]
    
}
"""

In [61]:
# Expected output
expected= "-- SET\n" + \
    "O1_set:= P1 | P3;\n" + \
    "O2_set:= P1 | P2;\n" + "O3_set:= P2;\n" + \
    "-- RESET\n" + \
    "O1_reset:= P2;\n" + \
    "O2_reset:= P2 | P3;\n" + "O3_reset:= P1 | P2;\n" + \
    "-- OUTPUT\n" + \
    "O1:= O1_set & !O1_reset;\n" + \
    "O2:= O2_set & !O2_reset;\n" + "O3:= O3_set & !O3_reset;\n"

In [4]:
# Convert Json string to python dictionary
input_dict= json.loads(input_str, object_pairs_hook=OrderedDict)


In [5]:
input_dict

OrderedDict([('P1', [['O1', 'O2'], ['O3']]),
             ('P2', [['O2', 'O3'], ['O1', 'O2']]),
             ('P3', [['O1'], ['O2', 'O3']])])

In [38]:
# Get set of outputs
output_set= set()

for key in list(input_dict):
    for output in input_dict[key][0]:
        output_set.add(output)
    for output in input_dict[key][1]:
        output_set.add(output)


In [39]:
# Sort alphabetically
output_set= sorted(output_set)

In [40]:
print(output_set)

['O1', 'O2', 'O3']


In [41]:
# Collate list of places for which each output is set/reset
output_dict= OrderedDict()

for output in output_set:
    # Initialize list of places for which output is set/reset
    output_dict[output]= [[],[]]

    for key in list(input_dict):
        # Add places for which output is set
        if output in input_dict[key][0]:
            output_dict[output][0].append(key)

        # Add places for which output is reset
        if output in input_dict[key][1]:
            output_dict[output][1].append(key)


In [43]:
print(output_dict)

OrderedDict([('O1', [['P1', 'P3'], ['P2']]), ('O2', [['P1', 'P2'], ['P2', 'P3']]), ('O3', [['P2'], ['P1', 'P3']])])


In [62]:
# Generate output string
set_output_str= "-- SET\n"
reset_output_str= "-- RESET\n"
output_str= "-- OUTPUT\n"

# Loop through output keys
for key in list(output_dict):
    output= output_dict[key]

    # Get set places
    set_output_str= set_output_str +  key + "_set:= "
    for idx_set,place_set in enumerate(output[0]):
        # Not last item
        if idx_set < len(output[0])-1:
            set_output_str= set_output_str + output[0][idx_set] + " | "
        # Last Item
        else:
            set_output_str= set_output_str + output[0][idx_set] + ";\n"

    # Get reset places
    reset_output_str= reset_output_str +  key + "_reset:= "
    for idx_reset,place_reset in enumerate(output[1]):
        # Not last item
        if idx_reset < len(output[1])-1:
            reset_output_str= reset_output_str + output[1][idx_reset] + " | "
        # Last Item
        else:
            reset_output_str= reset_output_str + output[1][idx_reset] + ";\n"

    # Compose output string
    output_str= output_str + key + ":= " + key + "_set & !" + key + "_reset;\n" 


In [66]:
set_output_str + reset_output_str + output_str

'-- SET\nO1_set:= P1 | P3;\nO2_set:= P1 | P2;\nO3_set:= P2;\n-- RESET\nO1_reset:= P2;\nO2_reset:= P2 | P3;\nO3_reset:= P1 | P3;\n-- OUTPUT\nO1:= O1_set + !O1_reset;\nO2:= O2_set + !O2_reset;\nO3:= O3_set + !O3_reset;\n'

In [67]:
expected

'-- SET\nO1_set:= P1 | P3;\nO2_set:= P1 | P2;\nO3_set:= P2;\n-- RESET\nO1_reset:= P2;\nO2_reset:= P2 | P3;\nO3_reset:= P1 | P2;\n-- OUTPUT\nO1:= O1_set & !O1_reset;\nO2:= O2_set & !O2_reset;\nO3:= O3_set & !O3_reset;\n'